# Structure your model

## Agenda
* Overall structure of a model in TensorFlow
* word2vec
* Name scope
* Embedding visualization

この時点までで、TensorFlowでの2つのシンプルなモデルを実装しました。
1つはシカゴの火災と窃盗の件数の線形回帰。
もう一つはロジスティック回帰を用いたMNISTのデータ・セットでの光学的な認識。
ツールを用いることで、より複雑なモデルのビルドができるようになりました。
しかし、複雑なモデルはより良い計画を要求します。もしくは、デバッグするのに少々厄介かもしれません。
次の2つの講義では、モデルの効果的な構成に関するディスカッションをします。
そして、word2vecの例を通じてそれを行います。

あなたが言葉の構造を最もよく精通し、wod2vecのようなモデルの重要性を理解している、とします。
あまり詳しくない型は[CS 224Nのスライド](http://web.stanford.edu/class/cs224n/lectures/cs224n-2017-lecture2.pdf)を参照しましょう。
オリジナルの論文は以下になります。
* [Distributed Representations of Words and Phrases and their Compositionality](https://arxiv.org/pdf/1301.3781.pdf)
* [Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/pdf/1301.3781.pdf)

端的に言うと、単語のベクトル表現を必要とします
それはいくつかのマジックのトリックを行うためにNeural Networkにinputする必要があります。多くのモデル学習の基礎から得られた単語のベクトルはCS224N学ぶことができました。現実世界での言語モデルと同様に。

3D空間に単語のベクトル表現を描画することもできました。

先の講義では、word2vecをビルドすることを試みました。skip-gramモデルを使用して。
ここからskip-gram モデルの説明とチュートリアルを見つけることができます。
[Word2Vec Tutorial - The Skip-Gram Model](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/)

skip-gramのモデルでは、単語のベクトル表現を求めるために、特定のタスクを実行するための単層隠れ層を持ったシンプルなニューラルネットワーク訓練します。しかし、そこでは私達は訓練されたタスクのために、ニューラルネットワークを使用しません。
その代わり、隠れ層の重みを気にします。これらの重みは学習するためのword vectors もしくは embedding matrixです。

私達がモデルとして学習しようとしている特定のフェークのタスクは与えられた中心の単語の近所を予測しています。文中の特定の単語を与えた場合、近くの単語を確認し、ランダムに1つピックアップします。ネットワークは私達が選択した近所の単語の語彙のすべての単語に対する可能性を教えてくれます。

### Softmax, Negative Sampling, and Noise Contrastive Estimation

CS 224Nでは、2つの訓練のメソッドを学びました。hierarchical softmaxとnegative samplingです。私達はsoftmaxを排除しました。それは正規化係数の計算コストが大きすぎるからです。そして、skip-gramのnegative samplingを学びました。

Negative samplingは、その名の通り、サンプリングに基づくアプローチの有名なものです。Negative samplingはNoise Contrastive Estimation(NCE)と呼ばれるアプローチの簡単なモデルです。

negative samplingはword embeddingsの学習に役立ちます。
Word Embeddingとは、単語間の意味上の関係性を捉えることです。

NCE は論理保証が良い。

Samplingベースのアプローチは、negative samplingかNCEに関わらず、訓練に時間がかかります。

### データセットに関して
text8 は 100MBのクリーンなテキストです。これは英語のWikipediaからdumpしたものです。クリーンなテキストを使用することで、生のテキストを処理する時間をTensorFlowに集中させることができます。

100MBは本当に良い word embeddingには十分ではありませんが、興味深い関係性を見るには十分です。17,005,207のトークンがスペースで区切られています。

良い結果を得るために、wikipediaのdumpの最初の10^9byteのfil9データセットを使用すべきです。これは[Matt Mahoney’s website](https://cs.fit.edu/~mmahoney/compression/textdata.html) で言及されています。

## どのようにしてTensorFlowのモデルを構築するか
たった2つのモデルしか触っていないし、それらは似たような構造をしている。

#### Phase1: グラフを組み立てる
1. inputとoutputのplaceholderを定義する
2. weightを定義する
3. 推論モデルを定義する
4. loss functionを定義する
5. optimizerを定義する

#### Phase2: 計算を実行する
モデルの基本的な訓練のために、いくつかのステップを踏みます。
1. 最初にすべてのvariablesを初期化します
2. 訓練用データを与えます。データサンプルの順序はランダムかもしれません。
3. 訓練データの推論モデルを実行します。
4. コストを計算します
5. モデルパラメータを調整します。


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector

from process_data import process_data

VOCAB_SIZE = 50000
BATCH_SIZE = 128
EMBED_SIZE = 128 # dimension of the word embedding vectors
SKIP_WINDOW = 1 # the context window
NUM_SAMPLED = 64    # Number of negative examples to sample.
LEARNING_RATE = 1.0
NUM_TRAIN_STEPS = 10000
SKIP_STEP = 2000 # how many steps to skip before reporting the loss

def word2vec(batch_gen):
    """ Build the graph for word2vec model and train it """
    # Step 1: define the placeholders for input and output
    # center_words have to be int to work on embedding lookup
    center_words = tf.placeholder(tf.int32, shape=[BATCH_SIZE], name='center_words')
    target_words = tf.placeholder(tf.int32, shape=[BATCH_SIZE, 1], name='target_words')

    # Step 2: define weights. In word2vec, it's actually the weights that we care about
    # vocab size x embed size
    # initialized to random uniform -1 to 1
    embed_matrix = tf.Variable(tf.random_uniform([VOCAB_SIZE, EMBED_SIZE], -1.0, 1.0), 
                            name='embed_matrix')

    # Step 3: define the inference
    # get the embed of input words using tf.nn.embedding_lookup
    # embed = tf.nn.embedding_lookup(embed_matrix, center_words, name='embed')

    embed = tf.nn.embedding_lookup(embed_matrix, center_words, name='embed')

    # Step 4: construct variables for NCE loss
    # tf.nn.nce_loss(weights, biases, labels, inputs, num_sampled, num_classes, ...)
    # nce_weight (vocab size x embed size), intialized to truncated_normal stddev=1.0 / (EMBED_SIZE ** 0.5)
    # bias: vocab size, initialized to 0
    nce_weight = tf.Variable(tf.truncated_normal([VOCAB_SIZE, EMBED_SIZE],
                                                stddev=1.0 / (EMBED_SIZE ** 0.5)), 
                                                name='nce_weight')
    nce_bias = tf.Variable(tf.zeros([VOCAB_SIZE]), name='nce_bias')


    # define loss function to be NCE loss function
    # tf.nn.nce_loss(weights, biases, labels, inputs, num_sampled, num_classes, ...)
    # need to get the mean accross the batch
    loss = tf.reduce_mean(tf.nn.nce_loss(weights=nce_weight, 
                                        biases=nce_bias, 
                                        labels=target_words, 
                                        inputs=embed, 
                                        num_sampled=NUM_SAMPLED, 
                                        num_classes=VOCAB_SIZE), name='loss')


    # Step 5: define optimizer
    optimizer = tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(loss)
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        total_loss = 0.0 # we use this to calculate the average loss in the last SKIP_STEP steps
        writer = tf.summary.FileWriter('./my_graph/no_frills/', sess.graph)
        for index in range(NUM_TRAIN_STEPS):
            centers, targets = next(batch_gen)
            loss_batch, _ = sess.run([loss, optimizer], feed_dict = {center_words: centers, target_words: targets})
            total_loss += loss_batch
            if (index + 1) % SKIP_STEP == 0:
                print('Average loss at step {}: {:5.1f}'.format(index, total_loss / SKIP_STEP))
                total_loss = 0.0
        writer.close()

def main():
    batch_gen = process_data(VOCAB_SIZE, BATCH_SIZE, SKIP_WINDOW)
    word2vec(batch_gen)

if __name__ == '__main__':
    main()

In [ ]:
""" word2vec with NCE loss 
and code to visualize the embeddings on TensorBoard
"""

# from __future__ import absolute_import
# from __future__ import division
# from __future__ import print_function

import os

import numpy as np
from tensorflow.contrib.tensorboard.plugins import projector
import tensorflow as tf

from process_data import process_data

VOCAB_SIZE = 50000
BATCH_SIZE = 128
EMBED_SIZE = 128 # dimension of the word embedding vectors
SKIP_WINDOW = 1 # the context window
NUM_SAMPLED = 64    # Number of negative examples to sample.
LEARNING_RATE = 1.0
NUM_TRAIN_STEPS = 100000
WEIGHTS_FLD = 'processed/'
SKIP_STEP = 2000

class SkipGramModel:
    """ Build the graph for word2vec model """
    def __init__(self, vocab_size, embed_size, batch_size, num_sampled, learning_rate):
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.batch_size = batch_size
        self.num_sampled = num_sampled
        self.lr = learning_rate
        self.global_step = tf.Variable(0, dtype=tf.int32, trainable=False, name='global_step')

    def _create_placeholders(self):
        """ Step 1: define the placeholders for input and output """
        with tf.device('/cpu:0'):
            with tf.name_scope("data"):
                self.center_words = tf.placeholder(tf.int32, shape=[self.batch_size], name='center_words')
                self.target_words = tf.placeholder(tf.int32, shape=[self.batch_size, 1], name='target_words')

    def _create_embedding(self):
        """ Step 2: define weights. In word2vec, it's actually the weights that we care about """
        # Assemble this part of the graph on the CPU. You can change it to GPU if you have GPU
        with tf.device('/cpu:0'):
            with tf.name_scope("embed"):
                self.embed_matrix = tf.Variable(tf.random_uniform([self.vocab_size, 
                                                                self.embed_size], -1.0, 1.0), 
                                                                name='embed_matrix')

    def _create_loss(self):
        """ Step 3 + 4: define the model + the loss function """
        with tf.device('/cpu:0'):        
            with tf.name_scope("loss"):
                # Step 3: define the inference
                embed = tf.nn.embedding_lookup(self.embed_matrix, self.center_words, name='embed')

                # Step 4: define loss function
                # construct variables for NCE loss
                nce_weight = tf.Variable(tf.truncated_normal([self.vocab_size, self.embed_size],
                                                            stddev=1.0 / (self.embed_size ** 0.5)), 
                                                            name='nce_weight')
                nce_bias = tf.Variable(tf.zeros([VOCAB_SIZE]), name='nce_bias')

                # define loss function to be NCE loss function
                self.loss = tf.reduce_mean(tf.nn.nce_loss(weights=nce_weight, 
                                                    biases=nce_bias, 
                                                    labels=self.target_words, 
                                                    inputs=embed, 
                                                    num_sampled=self.num_sampled, 
                                                    num_classes=self.vocab_size), name='loss')
    def _create_optimizer(self):
        """ Step 5: define optimizer """
        with tf.device('/cpu:0'):        
            self.optimizer = tf.train.GradientDescentOptimizer(self.lr).minimize(self.loss)

    def _create_summaries(self):
        with tf.name_scope("summaries"):
            tf.summary.scalar("loss", self.loss)
            tf.summary.histogram("histogram loss", self.loss)
            # because you have several summaries, we should merge them all
            # into one op to make it easier to manage
            self.summary_op = tf.summary.merge_all()

    def build_graph(self):
        """ Build the graph for our model """
        self._create_placeholders()
        self._create_embedding()
        self._create_loss()
        self._create_optimizer()
        self._create_summaries()

def train_model(model, batch_gen, num_train_steps, weights_fld):
    saver = tf.train.Saver() # defaults to saving all variables - in this case embed_matrix, nce_weight, nce_bias

    initial_step = 0
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        ckpt = tf.train.get_checkpoint_state(os.path.dirname('checkpoints/checkpoint'))
        # if that checkpoint exists, restore from checkpoint
        if ckpt and ckpt.model_checkpoint_path:
            saver.restore(sess, ckpt.model_checkpoint_path)

        total_loss = 0.0 # we use this to calculate late average loss in the last SKIP_STEP steps
        writer = tf.summary.FileWriter('improved_graph/lr' + str(LEARNING_RATE), sess.graph)
        initial_step = model.global_step.eval()
        for index in range(initial_step, initial_step + num_train_steps):
            centers, targets = next(batch_gen)
            feed_dict={model.center_words: centers, model.target_words: targets}
            print(type(centers))
            print(type(targets))
            loss_batch, _, summary = sess.run([model.loss, model.optimizer, model.summary_op],  feed_dict=feed_dict)
            writer.add_summary(summary, global_step=index)
            total_loss += loss_batch
            if (index + 1) % SKIP_STEP == 0:
                print('Average loss at step {}: {:5.1f}'.format(index, total_loss / SKIP_STEP))
                total_loss = 0.0
                saver.save(sess, 'checkpoints/skip-gram', index)
        
        ####################
        # code to visualize the embeddings. uncomment the below to visualize embeddings
        final_embed_matrix = sess.run(model.embed_matrix)
        
        # # it has to variable. constants don't work here. you can't reuse model.embed_matrix
        embedding_var = tf.Variable(final_embed_matrix[:1000], name='embedding')
        sess.run(embedding_var.initializer)

        config = projector.ProjectorConfig()
        summary_writer = tf.summary.FileWriter('processed')

        # # add embedding to the config file
        embedding = config.embeddings.add()
        embedding.tensor_name = embedding_var.name
        
        # # link this tensor to its metadata file, in this case the first 500 words of vocab
        embedding.metadata_path = 'processed/vocab_1000.tsv'

        # # saves a configuration file that TensorBoard will read during startup.
        projector.visualize_embeddings(summary_writer, config)
        saver_embed = tf.train.Saver([embedding_var])
        saver_embed.save(sess, 'processed/model3.ckpt', 1)

def main():
    model = SkipGramModel(VOCAB_SIZE, EMBED_SIZE, BATCH_SIZE, NUM_SAMPLED, LEARNING_RATE)
    model.build_graph()
    batch_gen = process_data(VOCAB_SIZE, BATCH_SIZE, SKIP_WINDOW)
    train_model(model, batch_gen, NUM_TRAIN_STEPS, WEIGHTS_FLD)

if __name__ == '__main__':
    main()